In [0]:
from pyspark.sql.functions import col
from pyspark.sql.types import IntegerType, DoubleType, BooleanType, DateType

In [0]:
configs = {"fs.azure.account.auth.type": "OAuth",
"fs.azure.account.oauth.provider.type": "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider",
"fs.azure.account.oauth2.client.id": "2f8bb546-b655-4410-afca-f23cd8078097",
"fs.azure.account.oauth2.client.secret": dbutils.secrets.get(scope="azure_kv_scope", key="secret-key-client"),
"fs.azure.account.oauth2.client.endpoint": "https://login.microsoftonline.com/7ff2a023-68d0-472e-be3f-570fdcf65ecb/oauth2/token"}

dbutils.fs.mount(
source = "abfss://paris-olympic-data@parisolympicdata.dfs.core.windows.net", # contrainer@storageacc
mount_point = "/mnt/parisolympic",
extra_configs = configs)

Out[3]: True

In [0]:
%fs
ls "/mnt/parisolympic"

path,name,size,modificationTime
dbfs:/mnt/parisolympic/raw-data/,raw-data/,0,1761910443000
dbfs:/mnt/parisolympic/transformed-data/,transformed-data/,0,1761910412000


In [0]:
spark

SparkSession - hive 
 
 
 SparkContext 

 Spark UI 

 
 Version 
 v3.3.2 
 Master 
 local[*, 4] 
 AppName 
 Databricks Shell

In [0]:
athletes = spark.read.format("csv").option("header","true").option("inferSchema","true").load("/mnt/parisolympic/raw-data/athletes.csv")
coaches = spark.read.format("csv").option("header","true").option("inferSchema","true").load("/mnt/parisolympic/raw-data/coaches.csv")
entriesgender = spark.read.format("csv").option("header","true").option("inferSchema","true").load("/mnt/parisolympic/raw-data/entriesgender.csv")
medals = spark.read.format("csv").option("header","true").option("inferSchema","true").load("/mnt/parisolympic/raw-data/medals.csv")
teams = spark.read.format("csv").option("header","true").option("inferSchema","true").load("/mnt/parisolympic/raw-data/teams.csv")

In [0]:
athletes.show()

+--------------------+--------------------+-------------------+
|                Name|                Team|         Discipline|
+--------------------+--------------------+-------------------+
|AABERG IVERSEN Sarah|             Denmark|           Handball|
|       AARDAHL Maren|              Norway|           Handball|
|     AARONSON Paxten|United States of ...|           Football|
|         AARTS Laura|         Netherlands|         Water Polo|
| AASMUL Victor Gairy|             Denmark|       Canoe Sprint|
|       AASTEDT Sofia|              Sweden|           Swimming|
|ABABAKIROV Samirkhon|          Kazakhstan|          Taekwondo|
|         ABAD Nestor|               Spain|Artistic Gymnastics|
|          ABADA Liel|              Israel|           Football|
|       ABADIA Adrian|               Spain|             Diving|
|   ABAGNALE Giovanni|               Italy|             Rowing|
|ABAKAR Ismail Doudai|               Qatar|          Athletics|
|  ABAKAROV Zelimkhan|             Alban

In [0]:
athletes.printSchema()

root
 |-- Name: string (nullable = true)
 |-- Team: string (nullable = true)
 |-- Discipline: string (nullable = true)



In [0]:
coaches.show()

+--------------------+--------------------+-----------------+-----+
|                Name|                Team|       Discipline|Event|
+--------------------+--------------------+-----------------+-----+
|        ABARCA Megan|United States of ...|Artistic Swimming| Duet|
|   ABDELFATTAH Aziza|               Egypt|Artistic Swimming| Team|
|        ADZIC Dragan|            Slovenia|         Handball|Women|
|ALLER CARBALLO Ma...|               Spain|       Basketball| null|
|  ALTENBURG Valentin|             Germany|           Hockey| null|
|AMOR Simon Daniel...|               Japan|     Rugby Sevens|  Men|
| ANNAN Alyson Regina|People's Republic...|           Hockey| null|
|      ANNEQUIN Marie|              France|Artistic Swimming| null|
|        ARKELL Robin|             Germany|           Hockey| null|
|  ARZA Carlos Vivier|              Angola|         Handball|Women|
|         AXNER Tomas|              Sweden|         Handball|Women|
|           BAM Jenna|        South Africa|     

In [0]:

coaches.printSchema()

root
 |-- Name: string (nullable = true)
 |-- Team: string (nullable = true)
 |-- Discipline: string (nullable = true)
 |-- Event: string (nullable = true)



In [0]:
entriesgender.show()

+--------------------+----+------+-----+
|          Discipline|Male|Female|Total|
+--------------------+----+------+-----+
|      3x3 Basketball|  32|    32|   64|
|             Archery|  64|    64|  128|
| Artistic Gymnastics|  96|    94|  190|
|   Artistic Swimming|   0|   106|  106|
|           Athletics|1036|   982| 2018|
|           Badminton|  88|    87|  175|
|          Basketball| 144|   144|  288|
|    Beach Volleyball|  48|    48|   96|
|              Boxing| 124|   124|  248|
|            Breaking|  16|    17|   33|
|        Canoe Slalom|  43|    41|   84|
|        Canoe Sprint| 119|   120|  239|
|Cycling BMX Frees...|  12|    12|   24|
|  Cycling BMX Racing|  34|    33|   67|
|Cycling Mountain ...|  36|    34|   70|
|        Cycling Road|  94|    94|  188|
|       Cycling Track| 117|   118|  235|
|              Diving|  67|    68|  135|
|          Equestrian| 149|    93|  242|
|             Fencing| 130|   130|  260|
+--------------------+----+------+-----+
only showing top

In [0]:
entriesgender.printSchema()

root
 |-- Discipline: string (nullable = true)
 |-- Male: integer (nullable = true)
 |-- Female: integer (nullable = true)
 |-- Total: integer (nullable = true)



In [0]:
medals.show()

+--------------------+----+------+------+-----+
|         Nationality|gold|silver|bronze|total|
+--------------------+----+------+------+-----+
|United States of ...|  40|    44|    42|  126|
|People's Republic...|  40|    27|    24|   91|
|               Japan|  20|    12|    13|   45|
|           Australia|  18|    19|    16|   53|
|              France|  16|    26|    22|   64|
|         Netherlands|  15|     7|    12|   34|
|       Great Britain|  14|    22|    29|   65|
|   Republic of Korea|  13|     9|    10|   32|
|               Italy|  12|    13|    15|   40|
|             Germany|  12|    13|     8|   33|
|         New Zealand|  10|     7|     3|   20|
|              Canada|   9|     7|    11|   27|
|          Uzbekistan|   8|     2|     3|   13|
|             Hungary|   6|     7|     6|   19|
|               Spain|   5|     4|     9|   18|
|              Sweden|   4|     4|     3|   11|
|               Kenya|   4|     2|     5|   11|
|              Norway|   4|     1|     3

In [0]:
medals.printSchema()

root
 |-- Nationality: string (nullable = true)
 |-- gold: integer (nullable = true)
 |-- silver: integer (nullable = true)
 |-- bronze: integer (nullable = true)
 |-- total: integer (nullable = true)



In [0]:
teams.show()

+--------------------+--------------------+--------------+----------+
|                Name|                 NOC|    Discipline|     event|
+--------------------+--------------------+--------------+----------+
|          Australia |          Australia |3x3 Basketball|     Women|
|         Azerbaijan |         Azerbaijan |3x3 Basketball|     Women|
|             Canada |             Canada |3x3 Basketball|     Women|
|             France |             France |3x3 Basketball|       Men|
|             France |             France |3x3 Basketball|     Women|
|            Germany |            Germany |3x3 Basketball|     Women|
|             Latvia |             Latvia |3x3 Basketball|       Men|
|          Lithuania |          Lithuania |3x3 Basketball|       Men|
|        Netherlands |        Netherlands |3x3 Basketball|       Men|
|People's Republic...|People's Republic...|3x3 Basketball|       Men|
|People's Republic...|People's Republic...|3x3 Basketball|     Women|
|             Poland

In [0]:
teams.printSchema()

root
 |-- Name: string (nullable = true)
 |-- NOC: string (nullable = true)
 |-- Discipline: string (nullable = true)
 |-- event: string (nullable = true)



Find the top countries with the highest number of gold medal in the competition

In [0]:
top_gold_medal_countries = medals.orderBy("Gold", ascending=False).select("Nationality","Gold").show()

+--------------------+----+
|         Nationality|Gold|
+--------------------+----+
|United States of ...|  40|
|People's Republic...|  40|
|               Japan|  20|
|           Australia|  18|
|              France|  16|
|         Netherlands|  15|
|       Great Britain|  14|
|   Republic of Korea|  13|
|               Italy|  12|
|             Germany|  12|
|         New Zealand|  10|
|              Canada|   9|
|          Uzbekistan|   8|
|             Hungary|   6|
|               Spain|   5|
|              Sweden|   4|
|               Kenya|   4|
|              Norway|   4|
|             Ireland|   4|
|              Brazil|   3|
+--------------------+----+
only showing top 20 rows



Calculation of the average number of entries by gender for each discipline

In [0]:
average_entries_by_gender = entriesgender.withColumn(
    'Avg_Female', entriesgender['Female'] / entriesgender['Total']
).withColumn(
    'Avg_Male', entriesgender['Male'] / entriesgender['Total']
)
average_entries_by_gender.show()

+--------------------+----+------+-----+-------------------+-------------------+
|          Discipline|Male|Female|Total|         Avg_Female|           Avg_Male|
+--------------------+----+------+-----+-------------------+-------------------+
|      3x3 Basketball|  32|    32|   64|                0.5|                0.5|
|             Archery|  64|    64|  128|                0.5|                0.5|
| Artistic Gymnastics|  96|    94|  190|0.49473684210526314| 0.5052631578947369|
|   Artistic Swimming|   0|   106|  106|                1.0|                0.0|
|           Athletics|1036|   982| 2018|0.48662041625371655| 0.5133795837462835|
|           Badminton|  88|    87|  175|0.49714285714285716| 0.5028571428571429|
|          Basketball| 144|   144|  288|                0.5|                0.5|
|    Beach Volleyball|  48|    48|   96|                0.5|                0.5|
|              Boxing| 124|   124|  248|                0.5|                0.5|
|            Breaking|  16| 

In [0]:
athletes.repartition(1).write.mode("overwrite").option("header",'true').csv("/mnt/parisolympic/transformed-data/athletes")
coaches.repartition(1).write.mode("overwrite").option("header","true").csv("/mnt/parisolympic/transformed-data/coaches")
entriesgender.repartition(1).write.mode("overwrite").option("header","true").csv("/mnt/parisolympic/transformed-data/entriesgender")
medals.repartition(1).write.mode("overwrite").option("header","true").csv("/mnt/parisolympic/transformed-data/medals")
teams.repartition(1).write.mode("overwrite").option("header","true").csv("/mnt/parisolympic/transformed-data/teams")
     